In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import os
import urllib.request
import tarfile

urllib.request.urlretrieve('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz', 'dataset.gz')
with tarfile.open('dataset.gz', 'r:gz') as tar:
    tar.extractall()

In [0]:
path = "aclImdb/{}/{}/"
trainfils, testfils = [], []
for g,collection in {'train': trainfils, 'test': testfils}.items() :
  for i,p in enumerate(['neg','pos']) :
    folder = path.format(g,p)
    for name in os.listdir(folder) :
      record = {
          'name' : name,
          'text' : open(folder+name).read(),
          'label' : i
      }
      collection.append(record)

import pandas as pd

train_df = pd.DataFrame(trainfils)
test_df = pd.DataFrame(testfils)

In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import re
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
tqdm.pandas()

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

MAX_LEN = 128
def make_clean(s) :
  s = cleanhtml(s)
  for i in range(10) :
    s = s.replace(str(i), ' ')
  tokens = np.array(tokenizer.tokenize(s.lower()))
  tokens = tokens[~np.isin(tokens, stopwords.words())]
  return ' '.join(tokens)

train_df['clean'] = train_df['text'].progress_apply(make_clean)
test_df['clean'] = test_df['text'].progress_apply(make_clean)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
train_df.head()

,name,text,label,clean
0,111_4.txt,My interest in Dorothy Stratten caused me to p...,0,interest dorothy stratten caused purchase vide...
1,1576_1.txt,"this film has it all; the deft camera work, re...",0,film deft camera work reminiscent martin scorc...
2,10577_2.txt,Some of the filmmakers who are participating i...,0,filmmakers participating series made really gr...
3,10090_1.txt,"This movie is a joke. I mean a ""ha ha"" funny j...",0,movie joke mean funny joke redeeming thing goo...
4,5188_1.txt,"TV pilots, don't you love them? Quinn Martin t...",0,tv pilots love quinn martin tried successful b...


In [0]:
import numpy as np

MAX_LEN = 128

# Create datasets (Only take up to MAX_LEN words)
train_text = train_df['clean'].tolist()
train_text = [' '.join(t.split()[0:MAX_LEN]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_df['label'].tolist()
test_text = test_df['clean'].tolist()
test_text = [' '.join(t.split()[0:MAX_LEN]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['label'].tolist()

In [7]:
from keras.engine.topology import Layer
import tensorflow_hub as hub
from keras import backend as K

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)
    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result
    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

Using TensorFlow backend.


In [0]:
class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

In [9]:
from keras import layers
from keras.models import Model
from keras.optimizers import Adam

input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = ElmoEmbeddingLayer()(input_text)
densembd = layers.Dense(1024, activation='relu')(embedding)
nmsk = NonMasking()(densembd)
rshp = layers.Reshape((-1,1024))(nmsk)
convolution = layers.Convolution1D(50, 3, padding='same', activation='relu')(rshp)
convolution = layers.GlobalMaxPooling1D()(convolution)
dense = layers.Dense(50, activation='relu')(convolution)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4), metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
non_masking_1 (NonMasking)   (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1024)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 50)             153650    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0   

In [0]:
from tensorflow.compat.v1 import keras

session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

In [0]:
import keras
import numpy as np
import sklearn.metrics as sklm

class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.accuracy = []
        self.auc = []

    def on_epoch_end(self, epoch, logs={}):
        score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]
        self.loss.append(keras.losses.binary_crossentropy(tf.convert_to_tensor(targ), tf.convert_to_tensor(predict)).eval(session=session).mean())
        self.auc.append(sklm.roc_auc_score(targ, score))
        self.precision.append(sklm.precision_score(targ, predict))
        self.recall.append(sklm.recall_score(targ, predict))
        self.f1s.append(sklm.f1_score(targ, predict))
        self.accuracy.append(sklm.accuracy_score(targ, predict))

        pd.DataFrame({
            'loss': self.loss,
            'precision': self.precision,
            'recall': self.recall,
            'f1s': self.f1s,
            'accuracy': self.accuracy,
            'auc': self.auc
        }).to_csv('recors.csv')

        return

metrics = Metrics()

In [12]:
model.fit(train_text,
          train_label,
          validation_data=(test_text, test_label),
          epochs=50,
          callbacks=[metrics],
          batch_size=32)

Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 805s 32ms/step - loss: 0.4556 - accuracy: 0.7859 - val_loss: 0.4033 - val_accuracy: 0.8158
Epoch 2/50
25000/25000 [==============================] - 789s 32ms/step - loss: 0.4017 - accuracy: 0.8181 - val_loss: 0.3835 - val_accuracy: 0.8245
Epoch 3/50
25000/25000 [==============================] - 789s 32ms/step - loss: 0.3808 - accuracy: 0.8290 - val_loss: 0.3801 - val_accuracy: 0.8281
Epoch 4/50
25000/25000 [==============================] - 788s 32ms/step - loss: 0.3729 - accuracy: 0.8323 - val_loss: 0.3997 - val_accuracy: 0.8185
Epoch 5/50
25000/25000 [==============================] - 787s 31ms/step - loss: 0.3635 - accuracy: 0.8374 - val_loss: 0.3878 - val_accuracy: 0.8250
Epoch 6/50
25000/25000 [==============================] - 794s 32ms/step - loss: 0.3517 - accuracy: 0.8448 - val_loss: 0.3689 - val_accuracy: 0.8353
Epoch 7/50
25000/25000 [==============================] 

KeyboardInterrupt: ignored